In [28]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd

In [31]:
def A(pN,pM, Xi,m):
    p1 = pN-Xi*(pM-pN)
    pm = pM+Xi*(pM-pN)
    A = [p1]
    for i in range (1,m):
        new = p1 + (i-1)*(pm-p1)/(m-1)
        A.append(new)
    return A

In [50]:
A = A(1.78,2,0.1,15)

In [51]:
len(A)

15

In [40]:
def S(A): 
    S = []
    for i in range (len(A)):
        for j in range (len(A)): 
            S.append(s(A[i],A[j]))
    return S
    

In [52]:
S = S(A)

In [53]:
s_ex = S[100]

In [54]:
def pi(s): 
    return (s[0]-1)*np.exp(4*(2-s[0]))/(np.exp(4*(2-s[0]))+np.exp(4*(2-s[1]))-1)

In [48]:
pi(s_ex)

0.7188056694679223

In [55]:
Q = np.zeros((len(A),len(S)))

In [56]:
Q

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
def initQ()

In [42]:
len(S(p))

225

In [ ]:
#Créer une fonction qui trouve le maximum de Q(s,a)
def trouve_max(Q,s_prime):
    sol = Q[0][s_prime]
    for i in range (1,len(Q)): 
        if sol <= Q[i][s_prime]:
            sol = Q[i][s_prime]
    return sol        

In [ ]:
trouve_max(Q,1)

8

In [ ]:
def updateQ(Q,a_t,s_t, alpha, delta, pi_t, s_t1):
    Qbis = Q
    for i in range(len(Q)): 
        for j in range(len(Q[0])):
            if i == a_t and j == s_t: #faire attention à différencier valeurs et indices : ce n'est pas le cas ici 
                Qbis[i][j]= (1-alpha)*Q[i][j] + alpha*(pi_t + delta*trouve_max(Q,s_t1))

    return Qbis

In [ ]:
updateQ(Q,1,1, 0.5, 0.15, 1, 2)

array([[1, 2, 3],
       [4, 2, 6],
       [7, 8, 9]])